***Descargando las librerías***




In [25]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
banco = pd.read_csv('https://raw.githubusercontent.com/AndreaRubianoM/proyecto/master/data/bank-additional-full.csv', sep = ';')
banco.head()



,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


No tenemos variables con campos vacíos.

In [7]:
banco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [8]:
banco.isna().sum().sort_values()

age               0
euribor3m         0
cons.conf.idx     0
cons.price.idx    0
emp.var.rate      0
poutcome          0
previous          0
pdays             0
campaign          0
nr.employed       0
duration          0
month             0
contact           0
loan              0
housing           0
default           0
education         0
marital           0
job               0
day_of_week       0
y                 0
dtype: int64

Para un mejor entendimiento de las variables cambiaremos sus nombres.

In [9]:
banco2 = banco.rename(columns = {'age':'Edad', 'job':'Activ_económica', 'marital':'Estado_civil', 'education':'Nivel_educativo', 'default':'Mora_actual',
                                 'housing':'Créd_vivienda','loan':'Créd_personal', 'contact':'Medio_contacto', 'month':'Mes_contacto', 'day_of_week':'Día_contacto',
                                 'duration':'Duración_últcontacto', 'campaign':'Contactos_vigente', 'pdays':'Días_últcontacto', 'previous':'Contactos_anterior',
                                 'poutcome':'Resultado_anterior', 'emp.var.rate':'Var_empleo', 'cons.price.idx':'Indice_precios','cons.conf.idx':'Indice_confianza',
                                 'euribor3m':'Euribor3m', 'nr.employed':'Nro_Empleados', 'y':'Acepta_CDT'})
banco2.head()

,Edad,Activ_económica,Estado_civil,Nivel_educativo,Mora_actual,Créd_vivienda,Créd_personal,Medio_contacto,Mes_contacto,Día_contacto,...,Contactos_vigente,Días_últcontacto,Contactos_anterior,Resultado_anterior,Var_empleo,Indice_precios,Indice_confianza,Euribor3m,Nro_Empleados,Acepta_CDT
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no



***Eliminación variables de acuerdo a análisis***

In [11]:
banco3 = banco2.drop(columns=['Días_últcontacto','Contactos_anterior', 'Nro_Empleados', 'Var_empleo'])
banco3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Edad                  41188 non-null  int64  
 1   Activ_económica       41188 non-null  object 
 2   Estado_civil          41188 non-null  object 
 3   Nivel_educativo       41188 non-null  object 
 4   Mora_actual           41188 non-null  object 
 5   Créd_vivienda         41188 non-null  object 
 6   Créd_personal         41188 non-null  object 
 7   Medio_contacto        41188 non-null  object 
 8   Mes_contacto          41188 non-null  object 
 9   Día_contacto          41188 non-null  object 
 10  Duración_últcontacto  41188 non-null  int64  
 11  Contactos_vigente     41188 non-null  int64  
 12  Resultado_anterior    41188 non-null  object 
 13  Indice_precios        41188 non-null  float64
 14  Indice_confianza      41188 non-null  float64
 15  Euribor3m          

***Separación Variable Objetivo ***

In [12]:
X = banco3.drop(columns=['Acepta_CDT'])
y = banco3['Acepta_CDT']

pd.value_counts(y)

no     36548
yes     4640
Name: Acepta_CDT, dtype: int64

***Transformación de variables***

In [14]:
categoric = X.select_dtypes(include = ['object']).columns.to_list()
numeric = X.select_dtypes(include = ['int64', 'float64']).columns.to_list()

tf = ColumnTransformer([('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False), categoric),
                        ('scaler', StandardScaler(), numeric)], remainder='passthrough')

***División dataset***

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, shuffle=True, random_state=123)

In [17]:
X_trainPr = tf.fit_transform(X_train)
X_testPr = tf.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


***Entrenamiento Modelos***

***Regresión Logística***

In [18]:
modeloRL = LogisticRegression(C=0.1, class_weight='balanced', random_state=123)
modeloRL.fit(X_trainPr, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=0.1, class_weight='balanced', random_state=123)

In [21]:
predictions1 = modeloRL.predict(X_testPr)
print(confusion_matrix(y_test, predictions1))
print(classification_report(y_test, predictions1))

[[9412 1553]
 [ 170 1222]]
              precision    recall  f1-score   support

          no       0.98      0.86      0.92     10965
         yes       0.44      0.88      0.59      1392

    accuracy                           0.86     12357
   macro avg       0.71      0.87      0.75     12357
weighted avg       0.92      0.86      0.88     12357



***SVC***

In [23]:
modeloSVC = SVC(C=0.5, class_weight='balanced', random_state=123)
modeloSVC.fit(X_trainPr, y_train)

SVC(C=0.5, class_weight='balanced', random_state=123)

In [24]:
predictions2 = modeloSVC.predict(X_testPr)
print(confusion_matrix(y_test, predictions2))
print(classification_report(y_test, predictions2))

[[9198 1767]
 [ 104 1288]]
              precision    recall  f1-score   support

          no       0.99      0.84      0.91     10965
         yes       0.42      0.93      0.58      1392

    accuracy                           0.85     12357
   macro avg       0.71      0.88      0.74     12357
weighted avg       0.92      0.85      0.87     12357

